# RAG Agent Creation Tutorial

## Overview
This notebook demonstrates how to create a **Retrieval-Augmented Generation (RAG) agent** using the Hyland Agent Platform API. You'll learn to:

1. **Authenticate** with the Hyland API using OAuth 2.0 client credentials
2. **Configure** a RAG agent with specific parameters
3. **Create** the agent via REST API calls
4. **Understand** the response and next steps

## Prerequisites
- Valid Hyland client credentials (`CLIENT_ID` and `CLIENT_SECRET`)
- Basic understanding of REST APIs and JSON
- Familiarity with Python (helpful but not required)

## Instructions
1. **Run cells sequentially** from top to bottom
2. **Read the explanations** before each code section
3. **Enter your credentials** when prompted (they will be hidden)
4. **Check the output** after each cell execution

---

## Step 1: Authenticate and Obtain an JWT Access Token

This cell imports the necessary Python libraries and handles authentication with the Hyland API.

**When you run this cell:**
1. You'll be prompted for your CLIENT_ID and CLIENT_SECRET (input is hidden)
2. An access token is automatically retrieved
3. You're ready to create your RAG agent!

In [ ]:
import json
import urllib
import urllib.parse
import urllib.request
import requests
from getpass import getpass

agent_id = None
client_id = getpass("Please Enter your CLIENT_ID: ")
client_secret = getpass("Please Enter your CLIENT_SECRET: ")

try:
    req = urllib.request.Request(
        "https://auth.iam.dev.experience.hyland.com/idp/connect/token", 
        data=urllib.parse.urlencode({
            "client_id": client_id,
            "grant_type": "client_credentials",
            "client_secret": client_secret,
            "scope": "hxp environment_authorization"
        }).encode("utf-8"), 
        headers={"Content-Type": "application/x-www-form-urlencoded"}, 
        method="POST")
    with urllib.request.urlopen(req) as response_from_auth:            
        response_data = json.loads(response_from_auth.read().decode("utf-8"))
        access_token = response_data.get("access_token")
        print("✅ Access token obtained successfully!")     
except Exception as e:
    print(f"❌ Failed to get token: {str(e)}")

## Step 2: Configure Your RAG Agent

The next cell defines the configuration for your RAG agent. This JSON agent_config specifies how your agent will behave.

**Key Configuration Options:**
- **`name`**: A friendly name for your agent ("DocumentHelper")
- **`description`**: What your agent does ("Document assistant")
- **`agentType`**: Set to "rag" for Retrieval-Augmented Generation
- **`limit`**: Maximum number of documents to retrieve (10)
- **`llm_model_id`**: The AI model to use (Amazon Nova Micro)
- **`system_prompt`**: Instructions that guide how the AI responds
- **`temperature`**: Controls creativity (0.7 = balanced)
- **`max_tokens`**: Maximum response length (4000 tokens)

**💡 Tip:** You can modify these values to customize your agent's behavior before running the cell.

In [4]:
agent_config = {
    "name": "DocumentHelper",
    "description": "Document assistant",
    "agentType": "rag",
    "notes": "Initial version - shows all configuration options",
    "config": {
        "filter_value": {},
        "limit": 10,
        "llm_model_id": "amazon.nova-micro-v1:0",
        "system_prompt": """
Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {query_str}
Answer:
""",
        "inference_config": {
            "temperature": 0.7,
            "max_tokens": 4000
        }
    }
}

## Step 3: Create Your RAG Agent

This is the main API call that creates your RAG agent on the Hyland platform.

**The response will include:**
- Your new agent's unique ID
- Confirmation of the configuration
- Any validation messages

In [ ]:
auth_headers ={
    'Content-Type': 'application/json',
    'Accept': 'application/json',
    'Authorization': f'Bearer {access_token}'
}

response = requests.post(
    "https://api.agents.ai.dev.experience.hyland.com/agent-platform/v1/agents", 
    json=agent_config, 
    headers=auth_headers)

print(f"Status Code: {response.status_code}")
print(f"Response: {response.text}")

if response.status_code == 201:
    try:
        agent_id = response_data = response.json().get("id")
        print(f"\n✅ Agent created successfully! Agent ID: {agent_id}")
    except Exception as e:
        print(f"\n❌ Error parsing response: {e}")
else:
    print(f"\n❌ Failed to create agent. Status: {response.status_code}")

## Step 4: Test Your RAG Agent with a Single Question

Now let's test your newly created RAG agent with a simple, single question. This is the most basic way to interact with your agent.

**What happens:**
1. Send a direct question to your agent
2. Agent searches through available documents 
3. Returns an answer based on retrieved information
4. Shows source documents used for the response

**Content Lake**
Sample Dataset available for use

1. SOCIAL_MEDIA_POLICY_AND_GUIDELINES_-_2022_-_10-6-2022.pdf
2. 8850_Form_for_HANNA_RIVERS__ID__2230_.pdf
3. Background_Check_for_JANE_HARPER.pdf
4. TE_Global_Policy_2020_1_.pdf
5. CORPORATE_BRANDING_PRACTICES_-_2023_-_5-2-2023.pdf
6. Cover_Letter_for_JANE_HARPER.pdf
7. Emergency_Medical_Form_for_HANNA_RIVERS__ID__2230_.pdf
8. Employee_Federal_Whistleblower_Protection_Notice_2023_Global_Doc_Policies.pdf
9. ESG_Policy_2023_Policies__4_.pdf
10. Global_Anti-Bribery_Anti-Corruption_Policy.pdf
11. Global_Public_Customer_Gift_Policy_Global_Doc_Policies.pdf
12. HR_-_Employee_Handbook_for_EMPLOYEE_HANDBOOK__2023_.pdf
13. HR_-_W-4_for_JANE_HARPER_-_8-1-2023.pdf
14. Human-Resources-Consultant-Agreement.pdf
15. Hyland_Code_of_Conduct_Global_Policies__2_.pdf
16. Hyland_DEIB_Policy_Global_Policies.pdf
17. Hyland_Employee_Handbook_US_Policies.pdf
18. I-9_Form_for_HANNA_RIVERS__ID__2230_.pdf
19. Internet-Usage-Policy.pdf
20. NO_SMOKING_POLICY_-_2023_-_8-1-2023.pdf
21. Offer_Letter_for_HANNA_RIVERS__ID__2230_.pdf
22. Offer_Letter_for_JANE_HARPER_-_8-1-2023.pdf
23. SAFE_WORK_PROCEDURES_-_2023_-_5-2-2023.pdf
24. SALES_PROCESS_PROCEDURES_-_2023_-_5-2-2023.pdf


**💡 This is different from Step 5** which will show conversational format with message history.

In [ ]:
# Single question payload
single_question = {
    "messages": [{"role": "user", "content": "What is the company's sick leave policy?"}],
    "filterValue": {}
}

# Make the invoke request
invoke_url = f"https://api.agents.ai.dev.experience.hyland.com/agent-platform/v1/agents/{agent_id}/versions/latest/invoke"

try:
    response = requests.post(invoke_url, json=single_question, headers=auth_headers)
    if response.status_code == 200:
        data = response.json()
        print(f"🤖 ANSWER: {data.get('response', 'No response found')}")
        
        # Show sources if available
        if 'sources' in data and data['sources']:
            print(f"📚 Sources: {len(data['sources'])} documents used")
        else:
            print("📚 No source documents found")
    else:
        print(f"❌ Request failed: {response.status_code}")
        
except Exception as e:
    print(f"❌ Error: {str(e)}")

## Step 5: Invoke Your RAG Agent with Conversation

Now that your agent is created, you can test it using a conversational format with message history! This simulates a real conversation where the agent can reference previous exchanges.

**Conversation Format:**
- **`messages`**: Array of conversation turns with roles and content
- **`role`**: Either "user" (human) or "assistant" (AI agent)
- **`content`**: The actual message text
- **`filterValue`**: Additional filtering criteria (can be empty)

**How it works:**
1. Agent processes the entire conversation history
2. Uses context from previous messages to inform responses  
3. Searches relevant documents based on the latest user question
4. Provides contextually aware answers

**💡 Example Conversation:**
- User asks about sick leave policy
- Agent responds with policy details
- User follows up asking for more HR documents
- Agent understands the context and provides related information

In [ ]:
# Previous Messages (message history)
previous_messages = {
    "messages": [
        {
            "role": "user",
            "content": "What is the company's sick leave policy?"
        },
        {
            "role": "assistant", 
            "content": "The company's sick leave policy allows employees to take a certain number of sick days per year. Please refer to the employee handbook for specific details and eligibility criteria."
        },
        {
            "role": "user",
            "content": "Tell me more about HR docs?"
        }
    ],
    "filterValue": {}
}

# Make the invoke request
invoke_url = f"https://api.agents.ai.dev.experience.hyland.com/agent-platform/v1/agents/{agent_id}/versions/latest/invoke"

try:
    invoke_response = requests.post(invoke_url, json=previous_messages, headers=auth_headers)
    
    if invoke_response.status_code == 200:
        print("✅ Conversation processed successfully")
        data = response.json()
        print(f"🤖 ANSWER: {data.get('response', 'No response found')}")
    else:
        print(f"❌ Request failed: {invoke_response.status_code}")
        
except Exception as e:
    print(f"❌ Error: {str(e)}")

## Complete Workflow Summary

### 🎯 What You've Accomplished
You've successfully:
1. **Set up authentication** with the Hyland API and obtained access token
2. **Configured a RAG agent** with custom parameters
3. **Created the agent** on the platform
4. **Tested the agent** with a single question
5. **Invoked the agent** with conversational queries
6. **Processed responses** to understand the results

### ✅ If Everything Worked
Your RAG agent is now ready for production use! You can:
- **Integrate it into applications** using the agent ID
- **Scale up document processing** by uploading more content
- **Monitor performance** through the Hyland dashboard
- **Modify configuration** by creating new agent versions

---

## Troubleshooting Guide

### 🔧 Agent Creation Issues (Step 3)

**Status 201: Success** ✅
- Agent created successfully
- Agent ID extracted and ready for use

**Status 400: Bad Request** ❌
- Check your agent_config in Step 2
- Verify all required fields are present
- Ensure `llm_model_id` is valid

**Status 401: Unauthorized** ❌
- Verify your CLIENT_ID and CLIENT_SECRET in Step 1
- Check if credentials have expired
- Ensure you're using the correct environment

**Status 403: Forbidden** ❌
- Confirm your account has agent creation permissions
- Check if you're within usage limits

### 🤖 Agent Invocation Issues (Steps 4-5)

**Status 200: Success** ✅
- Query processed successfully
- Response contains agent's answer and sources

**Status 400: Bad Request** ❌
- Check your query format
- Ensure query is not empty
- Try a simpler question

**Status 404: Not Found** ❌
- Agent ID is incorrect or agent doesn't exist
- Verify Step 3 completed successfully
- Check if agent was deleted

**Status 401: Unauthorized** ❌
- Access token may have expired (tokens last ~1 hour)
- Re-run Step 1 to get a new token
- Check if headers are properly formatted

### 🔄 Testing Different Configurations

**To modify your agent:**
1. Go back to **Step 2** and change the `agent_config` dictionary
2. Re-run Step 3 to create a new agent
3. Test with Steps 4-5

**Common modifications:**
- Change `temperature` for more/less creative responses
- Adjust `limit` to retrieve more/fewer documents
- Modify `system_prompt` to change response style
- Try different `llm_model_id` options

### 🔄 Asking Multiple Questions

After completing all steps, you can:
1. **Rerun Steps 4-5** with different queries
2. **Change your questions** in Steps 4 or 5 and see different responses
3. **Compare single vs. conversational responses** to understand your agent's capabilities

### 📚 Additional Resources
- [Hyland Agent Platform Documentation](https://docs.hyland.com)
- [RAG Agent Configuration Guide](https://docs.hyland.com/agents/rag)
- [API Reference](https://api.agents.ai.dev.experience.hyland.com/docs)
- [Best Practices for RAG Prompting](https://docs.hyland.com/agents/prompting)